### Web Scraper
#### Scrape Instagram Image Captions, Likes and Comments

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np


username='natgeo'

browser = webdriver.Chrome()
browser.get('https://www.instagram.com/'+username+'/?hl=en')
Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")


# Scroll the page to load 100 images :

lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
i=0
links=[]

#might have to load initial images

while(match==False):
     lastCount = lenOfPage
     time.sleep(3)
     lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
     i= i+1
     source = browser.page_source
     data=bs(source, 'html.parser')
     body = data.find('body')
     script = body.find('span')
     for link in script.findAll('a'):
          #time.sleep(2)
          if re.match("/p", link.get('href')):
             links.append('https://www.instagram.com'+link.get('href'))
     if i>30 :
          match=True
           
print(links)
df_nat = pd.DataFrame(columns = ['display_url','caption','is_video','likes','comments']) 
        
result=pd.DataFrame()
for i in range(len(links)):
    try:
        time.sleep(2)
        page = urlopen(links[i]).read()
        data=bs(page, 'html.parser')
        body = data.find('body')
        script = body.find('script')
        raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
        json_data=json.loads(raw)
        posts =json_data['entry_data']['PostPage'][0]['graphql']
        posts= json.dumps(posts)
        posts = json.loads(posts)
        if(posts['shortcode_media']['is_video'] == False):
            display_url= posts['shortcode_media']['display_url'] 
            caption= posts['shortcode_media']['edge_media_to_caption']['edges'][0]['node']['text']
            is_video=posts['shortcode_media']['is_video'] 
            likes= posts['shortcode_media']['edge_media_preview_like']['count'] 
            comments= posts['shortcode_media']['edge_media_to_parent_comment']['count']
            #title= posts['shortcode_media']['title']
            df_nat.loc[len(df_nat)] = [display_url,caption,is_video,likes,comments]
          
       
    except:
        np.nan

df_nat.to_csv("C:/Users/Sahana/Downloads/nat_results1.csv")